In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_classification/clue/main')

Mounted at /content/gdrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 13.8MB/s 
     |████████████████████████████████| 890kB 50.9MB/s 
     |████████████████████████████████| 1.1MB 36.7MB/s 
     |████████████████████████████████| 2.9MB 54.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=24cb962f37640e9d039ac2f68b1629c3dd62fbabc7bfab73dfae80af309bdeac
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer, TFBertLMHeadModel
import os
import json
import time
import logging
import pprint
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import random

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Enabled: True


In [ ]:
params = {
  'pretrain_path': 'bert-base-chinese',
  'train_path': '../data/train.txt',
  'test_path': '../data/test.txt',
  'batch_size': 16,
  'max_len': 130,
  'buffer_size': 31728,
  'init_lr': 1e-5,
  'max_lr': 3e-5,
  'n_epochs': 4 * 10,
}

tokenizer = BertTokenizer.from_pretrained(params['pretrain_path'],
                                          lowercase = True,
                                          add_special_tokens = True)

In [ ]:
# stream data from text files
def data_generator(f_path, params):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = json.loads(line.rstrip())
      text, label = line['content'], line['label']
      text = list(text)
      text = ['[CLS]'] + text + ['[SEP]']
      text = tokenizer.convert_tokens_to_ids(text)
      text = [idx for idx in text if idx != 100]
      if len(text) > params['max_len']:
        _max_len = params['max_len'] // 2
        text = text[:_max_len] + text[-_max_len:]
      seg = [0] * len(text)
      
      noises = []
      labels_mask = []
      for idx in text:
        if (random.random() <= 0.15) and (idx != 101) and (idx != 102):
          dice = random.random()
          if dice <= 0.8:
            noises.append(103)
          elif dice <= 0.9:
            noises.append(idx)
          else:
            noises.append(random.randint(0, 21127))
          labels_mask.append(1)
        else:
          noises.append(idx)
          labels_mask.append(0)

      yield (noises, seg), (text, labels_mask)


def dataset(is_training, params):
  _shapes = (([None], [None]), ([None], [None]))
  _types = ((tf.int32, tf.int32), (tf.int32, tf.int32))
  _pads = ((0, 0), (0, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [ ]:
# input stream ids check
(text, seg), (labels, labels_mask) = next(data_generator(params['train_path'], params))
print(text)
print(seg)
print(labels)
print(labels_mask)

Reading ../data/train.txt
[101, 112, 872, 4761, 6887, 1914, 103, 1914, 7353, 6818, 3300, 784, 720, 1408, 8043, 1506, 1506, 3300, 4788, 103, 5456, 4696, 4638, 741, 677, 1091, 4638, 872, 1420, 1521, 872, 2157, 6929, 1779, 4788, 2357, 3221, 686, 4518, 677, 3297, 1920, 4638, 4788, 2357, 8024, 1506, 1506, 103, 7745, 872, 4638, 1568, 2124, 103, 6432, 2225, 1217, 2861, 10671, 4105, 2357, 3221, 7391, 4518, 677, 3297, 103, 4638, 4105, 2357, 1568, 1506, 1506, 1506, 112, 112, 4268, 4268, 8024, 1961, 4638, 103, 1355, 5456, 8013, 2769, 812, 1920, 2812, 7370, 3488, 2094, 6963, 103, 5436, 677, 3341, 2769, 4692, 1168, 3312, 1928, 5361, 7027, 103, 14509, 1355, 671, 2137, 103, 103, 166, 103, 1184, 1931, 1168, 4638, 103, 872, 6432, 3221, 679, 3221, 8043, 138, 4495, 4567, 140, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
model = TFBertLMHeadModel.from_pretrained(params['pretrain_path'],
                                          trainable = True,
                                          return_dict = True)
model.load_weights('../model/bert_further_pretrain.h5')

In [ ]:
step_size = 4 * params['buffer_size'] // params['batch_size']
decay_lr = tfa.optimizers.Triangular2CyclicalLearningRate(
  initial_learning_rate = params['init_lr'],
  maximal_learning_rate = params['max_lr'],
  step_size = step_size,)
optim = tf.optimizers.Adam(params['init_lr'])
global_step = 0

best_acc = 0.69
t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

# Baseline Accuracy
m = tf.keras.metrics.Accuracy()
for ((text, seg), (labels, labels_mask)) in dataset(is_training=False, params=params):
  logits = model([text, tf.sign(text), seg], training=False).logits
  m.update_state(
    y_true = labels,
    y_pred = tf.argmax(logits, -1),
    sample_weight = labels_mask,)
acc = m.result().numpy()
logger.info("Baseline Accuracy: {:.3f}".format(acc))

for _ in range(params['n_epochs']):
  # Training
  for ((text, seg), (labels, labels_mask)) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model([text, tf.sign(text), seg], training=True).logits
      loss = tf.compat.v1.losses.softmax_cross_entropy(
        onehot_labels = tf.one_hot(labels, 21128),
        logits = logits,
        weights = tf.cast(labels_mask, tf.float32),
        label_smoothing = .2,)
    
    trainable_vars = [v for v in model.trainable_variables if 'pooler' not in v.name]
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, trainable_vars)
    grads, _ = tf.clip_by_global_norm(grads, 5.)
    optim.apply_gradients(zip(grads, trainable_vars))
    
    if global_step % 100 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
        global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
  # Evaluation
  m = tf.keras.metrics.Accuracy()

  for ((text, seg), (labels, labels_mask)) in dataset(is_training=False, params=params):
    logits = model([text, tf.sign(text), seg], training=False).logits
    m.update_state(
      y_true = labels,
      y_pred = tf.argmax(logits, -1),
      sample_weight = labels_mask,)

  acc = m.result().numpy()
  logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))

  if acc > best_acc:
    best_acc = acc
    model.save_weights('../model/bert_further_pretrain.h5', save_format='h5')

  logger.info("Best Accuracy: {:.3f}".format(best_acc))

Reading ../data/test.txt
INFO:tensorflow:Baseline Accuracy: 0.692
Reading ../data/train.txt
INFO:tensorflow:Step 0 | Loss: 5.6769 | Spent: 52.0 secs | LR: 0.000010
INFO:tensorflow:Step 100 | Loss: 3.8050 | Spent: 53.5 secs | LR: 0.000010
INFO:tensorflow:Step 200 | Loss: 3.5434 | Spent: 54.6 secs | LR: 0.000011
INFO:tensorflow:Step 300 | Loss: 3.6904 | Spent: 55.3 secs | LR: 0.000011
INFO:tensorflow:Step 400 | Loss: 3.6488 | Spent: 55.3 secs | LR: 0.000011
INFO:tensorflow:Step 500 | Loss: 3.8685 | Spent: 55.4 secs | LR: 0.000011
INFO:tensorflow:Step 600 | Loss: 3.5431 | Spent: 54.7 secs | LR: 0.000012
INFO:tensorflow:Step 700 | Loss: 3.7322 | Spent: 55.2 secs | LR: 0.000012
INFO:tensorflow:Step 800 | Loss: 3.9790 | Spent: 54.4 secs | LR: 0.000012
INFO:tensorflow:Step 900 | Loss: 3.4762 | Spent: 55.2 secs | LR: 0.000012
INFO:tensorflow:Step 1000 | Loss: 3.5651 | Spent: 55.4 secs | LR: 0.000013
INFO:tensorflow:Step 1100 | Loss: 3.6787 | Spent: 55.1 secs | LR: 0.000013
INFO:tensorflow:Step

KeyboardInterrupt: ignored

In [ ]:
print(model.weights[5]) # for later check if the weight is correctly transferred to other task

<tf.Variable 'tf_bert_lm_head_model/bert/encoder/layer_._0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32, numpy=
array([[ 0.11825976,  0.01059594,  0.00478886, ..., -0.04938788,
         0.01636124,  0.01673737],
       [-0.00918018, -0.00905642, -0.00512347, ...,  0.02321002,
        -0.09011449, -0.03552252],
       [ 0.0065572 , -0.00295785,  0.03182212, ...,  0.00119713,
        -0.04689885,  0.00050256],
       ...,
       [ 0.00973585, -0.00485693,  0.10374205, ...,  0.06686458,
        -0.03649763,  0.09566212],
       [ 0.00776388,  0.04061504, -0.03333197, ...,  0.00960802,
         0.04296997, -0.02524005],
       [-0.08363243,  0.01113833, -0.00531893, ...,  0.07489353,
        -0.01407121,  0.01147269]], dtype=float32)>
